In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import scipy.stats as stats

import utils

import importlib
importlib.reload(utils)

import os

In [2]:
out_dir = './'

In [3]:
all_df = pd.DataFrame()

In [4]:
simulation = False

In [5]:
experiments = {'6b2b35f0-3a65-d0a8-cb49-b2b6e4041562-rounds-10-evidence-1-bit-experiment':'Large-Population-Low-Evidence',
              }
n_evidences = {'6b2b35f0-3a65-d0a8-cb49-b2b6e4041562-rounds-10-evidence-1-bit-experiment':1,
              }
part_probs = {'6b2b35f0-3a65-d0a8-cb49-b2b6e4041562-rounds-10-evidence-1-bit-experiment':0.6,
              }

In [6]:

for in_dir in experiments:
    
    n_evidence = n_evidences[in_dir]

    exp_name = experiments[in_dir]
    
    part_prob = part_probs[in_dir]

    data,fails,parts = utils.get_data(in_dir, n_evidence)    
        
    df = utils.get_parsed_data(data, fails, parts, part_prob)
    
    df['Experiment'] = exp_name   
    df['exp_id'] = in_dir
    df['nets'] = df['nets'].astype('str') + '-' + in_dir    
    
    all_df = pd.concat([all_df, df])
    

FileNotFoundError: File b'./data/6b2b35f0-3a65-d0a8-cb49-b2b6e4041562-rounds-10-evidence-1-bit-experiment/info.csv' does not exist

In [ ]:
all_df.index = range(len(all_df))
df = all_df

In [ ]:
for exp_name in set(df['Experiment']):
    
    sub = df[df['Experiment'] == exp_name]
    
    colors = np.array(['']*len(sub['probs']))
    colors[np.array(sub['probs']) > 0.5] = 'blue'
    colors[np.array(sub['probs']) <= 0.5] = 'red'
    
    
    sns.set(context = 'paper', font_scale = 2, font='serif', style = 'white',  rc={"lines.linewidth": 2.5})    
    fig, ax = plt.subplots()
    plt.scatter(sub['evidences'],sub['final_pops'],s=50,alpha=0.5,c=colors)
    plt.xlabel('Last Evidence')
    plt.ylabel('Popularity')
    #plt.title(exp_name.replace('-',' '))
    
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    ax.plot(lims, lims, 'k-', alpha=0.1, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    
    plt.savefig(out_dir + exp_name + 'pop-last-evidence.jpg', bbox_inches = 'tight')

    sns.set(context = 'paper', font_scale = 2, font='serif', style = 'white',  rc={"lines.linewidth": 2.5})    
    fig, ax = plt.subplots()
    plt.scatter(sub['net_evidences'],sub['final_pops'],s=50,alpha=0.5,c=colors)
    plt.xlabel('Total Evidence')
    plt.ylabel('Popularity')
    #plt.title(exp_name.replace('-',' '))
    
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    ax.plot(lims, lims, 'k-', alpha=0.1, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    
    plt.savefig(out_dir + exp_name + 'pop-net-evidence.jpg', bbox_inches = 'tight')    

    sns.set(context = 'paper', font_scale = 2, font='serif', style = 'white',  rc={"lines.linewidth": 2.5})    
    fig, ax = plt.subplots()    
    plt.scatter(sub['posteriors'],sub['final_pops'],s=50,alpha=0.5,c=colors)
    plt.xlabel('Normative Posterior')
    plt.ylabel('Popularity')
    #plt.title(exp_name.replace('-',' '))
    
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    ax.plot(lims, lims, 'k-', alpha=0.1, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    
    plt.savefig(out_dir + exp_name + 'pop-post.jpg', bbox_inches = 'tight')   
    
    print(exp_name)
    print('correlation with net evidence', np.corrcoef(sub['net_evidences'],sub['final_pops'])[0,1])
    print('correlation with last evidence', np.corrcoef(sub['evidences'],sub['final_pops'])[0,1])
    print('correlation with posterior', np.corrcoef(sub['posteriors'],sub['final_pops'])[0,1])
    
    print('diff from net evidence', np.mean(np.abs(sub['net_evidences'] - sub['final_pops'])**2))
    print('diff from last evidence', np.mean(np.abs(sub['evidences'] - sub['final_pops'])**2))
    print('diff from posterior', np.mean(np.abs(sub['posteriors'] - sub['final_pops'])**2))

In [ ]:
pd.DataFrame({'evidence':df['net_evidences'],'last_evidence':df['evidences'],'popularity':df['final_pops'], 'game':df['nets'], 'experiment':df['exp_id']}).to_csv('evidence-pops.csv')
os.system('Rscript analyze.R > tmp')
with open('tmp') as f:
    for l in f.readlines():
        print(l)